In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from scipy.stats import kurtosis, skew
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random
import re
import cv2
from tqdm.notebook import tqdm
import seaborn as sns
from collections import defaultdict
from scipy.stats import pearsonr
import scipy.stats as stats
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.model_selection import StratifiedKFold
from tqdm import trange
import math
import tqdm
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from scipy.stats import percentileofscore
from sklearn.metrics import roc_auc_score

In [ ]:
def get_top_n(list_a, num):
topid= sorted(range(len(list_a)),key= lambda i: list_a[i])[-num:]
array_a = np.array(list_a)
return list(array_a[topid]), topid

def seed_everything(seed: int = 42):
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
seed_everything(seed=0)
num_vector = 2048
first_selection = 10
ITERATION = 100

nice_path = 'NICE_Grading_of_experts_csv'
radiomics_path = 'Radiomics_feature_csv' 
dllayer_path = 'Deep_feature_csv'

In [ ]:
df_nice = pd.read_csv(nice_path)
df_dllayer = pd.read_csv(dllayer_path)
df_radiomics = pd.read_csv(radiomics_path)

df_nice_texture_1_experts = df_nice['texture1'].values # 1, 2 for HP and AD
df_nice_texture_2_experts = df_nice['texture2'].values
df_nice_vessel_1_experts = df_nice['vessel1'].values
df_nice_vessel_2_experts = df_nice['vessel2'].values

In [ ]:
## 1st selection 

corr_matrix1_valid = np.zeros(shape=(93,num_vector)) 
regg_matrix1_valid = np.zeros(shape=(93,num_vector)) 
for i in trange(93):
    for j in range(num_vector):
        radio = np.array(df_radiomics)[:,i]
        conv = np.array(df_dllayer)[:,j]
        rho, pval = stats.spearmanr(radio, conv)
        if pval < 0.05:
            if np.abs(rho) > 0.5:
                coefficients = np.polyfit(radio, conv,1)
                corr_matrix1_valid[i,j] = rho
                regg_matrix1_valid[i,j] = coefficients[0]

interest_ij = list()
for i in range(93):
    for j in range(num_vector):
        if (np.abs(corr_matrix1_valid[i,j]) > 0.5): 
            append_idx = [i,j]
            interest_ij.append(append_idx)

radio_select_list = []
dl_select_list = []
for i in range(len(interest_ij)):
    radio_select_list.append(interest_ij[i][0])
    dl_select_list.append(interest_ij[i][1])
    
radio_select_set = np.array(list(set(radio_select_list)))
dl_select_set = np.array(list(set(dl_select_list)))
dl_select_list = np.array(dl_select_list)
dl_select_set = sorted(list(set(dl_select_list)))
interest_ij_array = np.array(interest_ij)
interest_ij_array_c = np.array(interest_ij_c)

In [ ]:
## 2nd & 3rd selection

iteration_result = np.zeros(shape=(ITERATION,len(dl_select_set),5))    
for ii in trange(len(dl_select_set)):
    DL_value = np.array(df_dllayer)[:,ii] 
    Label_value = fisrt_label
    split_idx = np.arange(Label_value.shape[0])     
    for iteration in range(ITERATION):
        train_index, test_index = train_test_split(split_idx, test_size=0.5, random_state=iteration, shuffle=True)
        dl_layer_train, dl_layer_test = DL_value[train_index], DL_value[test_index]
        df_nice_texture_1_experts_train, df_nice_texture_1_experts_test = df_nice_texture_1_experts[train_index], df_nice_texture_1_experts[test_index]
        df_nice_texture_2_experts_train, df_nice_texture_2_experts_test = df_nice_texture_2_experts[train_index], df_nice_texture_2_experts[test_index]
        df_nice_vessel_1_experts_train, df_nice_vessel_1_experts_test = df_nice_vessel_1_experts[train_index], df_nice_vessel_1_experts[test_index]
        df_nice_vessel_2_experts_train, df_nice_vessel_2_experts_test = df_nice_vessel_2_experts[train_index], df_nice_vessel_2_experts[test_index]
        dl_layer_train = dl_layer_train.reshape(-1,1)
        dl_layer_test = dl_layer_test.reshape(-1,1)
        df_nice_texture_1_experts_train_reshape = df_nice_texture_1_experts_train.reshape(-1,1)
        df_nice_texture_2_experts_train_reshape = df_nice_texture_2_experts_train.reshape(-1,1)
        df_nice_vessel_1_experts_train_reshape = df_nice_vessel_1_experts_train.reshape(-1,1)
        df_nice_vessel_2_experts_3rain_reshape = df_nice_vessel_2_experts_train.reshape(-1,1)
        df_nice_texture_1_experts_test_reshape = df_nice_texture_1_experts_test.reshape(-1,1)
        df_nice_texture_2_experts_test_reshape = df_nice_texture_2_experts_test.reshape(-1,1)
        df_nice_vessel_1_experts_test_reshape = df_nice_vessel_1_experts_test.reshape(-1,1)
        df_nice_vessel_2_experts_test_reshape = df_nice_vessel_2_experts_test.reshape(-1,1)
        model = LinearRegression()
        model.fit(dl_layer_train, df_nice_texture_1_experts_train_reshape)
        iteration_result[iteration, ii, 0] = r2_score(df_nice_texture_1_experts_test_reshape, model.predict(dl_layer_test))
        model = LinearRegression()
        model.fit(dl_layer_train, df_nice_texture_2_experts_train_reshape)
        iteration_result[iteration, ii, 1] = r2_score(df_nice_texture_2_experts_test_reshape, model.predict(dl_layer_test))
        model = LinearRegression()
        model.fit(dl_layer_train, df_nice_vessel_1_experts_train_reshape)
        iteration_result[iteration, ii, 2] = r2_score(df_nice_vessel_1_experts_test_reshape, model.predict(dl_layer_test))
        model = LinearRegression()
        model.fit(dl_layer_train, df_nice_vessel_2_experts_train_reshape)
        iteration_result[iteration, ii, 3] = r2_score(df_nice_vessel_2_experts_test_reshape, model.predict(dl_layer_test))
        model = svm.SVC(gamma='scale')
        model.fit(dl_layer_train, label_y_train)

abb_performance_r2_test = np.zeros(shape = (len(dl_select_set),4))
for top_selection_2 in trange(len(dl_select_set)):
    texture_1_selected = np.zeros(shape = (len(dl_select_set),1))
    texture_2_selected = np.zeros(shape = (len(dl_select_set),1))
    vessel_1_selected = np.zeros(shape = (len(dl_select_set),1))
    vessel_2_selected = np.zeros(shape = (len(dl_select_set),1))
    for iteration in range(ITERATION):
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,0]),first_selection)  
        texture_1_selected[top_idx] = texture_1_selected[top_idx]+1
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,1]),first_selection)
        texture_2_selected[top_idx] = texture_2_selected[top_idx]+1
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,2]),first_selection)
        vessel_1_selected[top_idx] = vessel_1_selected[top_idx]+1
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,3]),first_selection)
        vessel_2_selected[top_idx] = vessel_2_selected[top_idx]+1
    top_num, top_idx_texture1 = get_top_n(list(texture_1_selected),top_selection_2+1)
    top_num, top_idx_texture2 = get_top_n(list(texture_2_selected),top_selection_2+1)
    top_num, top_idx_vessel1 = get_top_n(list(vessel_1_selected),top_selection_2+1)
    top_num, top_idx_vessel2 = get_top_n(list(vessel_2_selected),top_selection_2+1)
    top_idx_1 = top_idx_texture1
    top_idx_2 = top_idx_texture2
    top_idx_3 = top_idx_vessel1
    top_idx_4 = top_idx_vessel2
    kf = KFold(n_splits=3, random_state=iteration, shuffle=True)       
    for case in range(4): 
        B = eval('top_idx_'+str(case+1))
        Y_value = np.array(second_NBI_re_arr)[:,B] # DL
        split_idx = np.arange(Y_value.shape[0])
        r2_1, r2_2, r2_3, r2_4= [], [], [], []
        for i, (train_index, test_index) in enumerate(kf.split(split_idx)):
            hold_out_test_train = split_idx[train_index]
            hold_out_test_test = split_idx[test_index]
            Y_value_train = Y_value[hold_out_test_train,:]
            Y_value_test = Y_value[hold_out_test_test,:]
            df_nice_texture_1_experts_3_T_train = df_nice_texture_1_experts_3_T[hold_out_test_train]
            df_nice_texture_2_experts_3_T_train = df_nice_texture_2_experts_3_T[hold_out_test_train]
            df_nice_vessel_1_experts_3_T_train = df_nice_vessel_1_experts_3_T[hold_out_test_train]
            df_nice_vessel_2_experts_3_T_train = df_nice_vessel_2_experts_3_T[hold_out_test_train]
            df_nice_texture_1_experts_3_T_test = df_nice_texture_1_experts_3_T[hold_out_test_test]
            df_nice_texture_2_experts_3_T_test = df_nice_texture_2_experts_3_T[hold_out_test_test]
            df_nice_vessel_1_experts_3_T_test = df_nice_vessel_1_experts_3_T[hold_out_test_test]
            df_nice_vessel_2_experts_3_T_test = df_nice_vessel_2_experts_3_T[hold_out_test_test]
            model = LinearRegression()
            model.fit(Y_value_train, df_nice_texture_1_experts_3_T_train)
            r2_1.append(r2_score(df_nice_texture_1_experts_3_T_test, model.predict(Y_value_test)))
            model = LinearRegression()
            model.fit(Y_value_train, df_nice_texture_2_experts_3_T_train)
            r2_2.append(r2_score(df_nice_texture_2_experts_3_T_test, model.predict(Y_value_test)))
            model = LinearRegression()
            model.fit(Y_value_train, df_nice_vessel_1_experts_3_T_train)
            r2_3.append(r2_score(df_nice_vessel_1_experts_3_T_test, model.predict(Y_value_test)))
            model = LinearRegression()
            model.fit(Y_value_train, df_nice_vessel_2_experts_3_T_train)
            r2_4.append(r2_score(df_nice_vessel_2_experts_3_T_test, model.predict(Y_value_test)))
        if case == 0:
            abb_performance_r2_test[top_selection_2,case] = np.mean(r2_1)
        elif case == 1:
            abb_performance_r2_test[top_selection_2,case] = np.mean(r2_2)
        elif case == 2:
            abb_performance_r2_test[top_selection_2,case] = np.mean(r2_3)
        elif case == 3:
            abb_performance_r2_test[top_selection_2,case] = np.mean(r2_4)

best_case = [np.argmax(abb_performance_r2_test[:,0]), np.argmax(abb_performance_r2_test[:,1]), np.argmax(abb_performance_r2_test[:,2]), np.argmax(abb_performance_r2_test[:,3])]
best_case = list(np.array(best_case) + 1)
for i, top_selection_2 in enumerate(best_case):
    texture_1_selected = np.zeros(shape = (len(dl_select_set),1)) # 63 x 1
    texture_2_selected = np.zeros(shape = (len(dl_select_set),1))
    vessel_1_selected = np.zeros(shape = (len(dl_select_set),1))
    vessel_2_selected = np.zeros(shape = (len(dl_select_set),1))
    for iteration in range(ITERATION):
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,0]),first_selection) # 우선 상위 N개 select 
        texture_1_selected[top_idx] = texture_1_selected[top_idx]+1
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,1]),first_selection)
        texture_2_selected[top_idx] = texture_2_selected[top_idx]+1
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,2]),first_selection)
        vessel_1_selected[top_idx] = vessel_1_selected[top_idx]+1
        top_num, top_idx = get_top_n(list(iteration_result[iteration,:,3]),first_selection)
        vessel_2_selected[top_idx] = vessel_2_selected[top_idx]+1
    if i == 0:
        top_num, top_idx_texture1 = get_top_n(list(texture_1_selected),top_selection_2)
        top_idx_1 = top_idx_texture1
        top_val1 = texture_1_selected[top_idx_texture1]
    elif i == 1:
        top_num, top_idx_texture2 = get_top_n(list(texture_2_selected),top_selection_2)
        top_idx_2 = top_idx_texture2
        top_val2 = texture_2_selected[top_idx_texture2]
    elif i ==2:
        top_num, top_idx_vessel1 = get_top_n(list(vessel_1_selected),top_selection_2)
        top_idx_3 = top_idx_vessel1
        top_val3 = vessel_1_selected[top_idx_vessel1]
    else:
        top_num, top_idx_vessel2 = get_top_n(list(vessel_2_selected),top_selection_2)
        top_idx_4 = top_idx_vessel2
        top_val4 = vessel_2_selected[top_idx_vessel2]

In [ ]:
## Final fitting
df_nice_texture_1_experts = df_nice['texture1'].values # 1, 2 for HP and AD
df_nice_texture_2_experts = df_nice['texture2'].values
df_nice_vessel_1_experts = df_nice['vessel1'].values
df_nice_vessel_2_experts = df_nice['vessel2'].values

Y_value_texture_1 = np.array(df_dllayer)[:,top_idx_1]
Y_value_texture_2 = np.array(df_dllayer)[:,top_idx_2]
Y_value_vessel_1 = np.array(df_dllayer)[:,top_idx_3]
Y_value_vessel_2 = np.array(df_dllayer)[:,top_idx_4]

linear_texture1 = LinearRegression()
linear_texture1.fit(Y_value_texture_1, df_nice_texture_1_experts)
linear_texture2 = LinearRegression()
linear_texture2.fit(Y_value_texture_2, df_nice_texture_2_experts)
linear_vessel1 = LinearRegression()
linear_vessel1.fit(Y_value_vessel_1, df_nice_vessel_1_experts)
linear_vessel2 = LinearRegression()
linear_vessel2.fit(Y_value_vessel_2, df_nice_vessel_2_experts)